# Test the Model

In [ ]:
pip install 'git+https://github.com/facebookresearch/detectron2.git@main'


  Cloning https://github.com/facebookresearch/detectron2.git (to revision main) to /tmp/pip-req-build-2o_75nes
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-2o_75nes
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.2 MB/s eta 0:00:00


In [ ]:
import detectron2
print("✅ Detectron2 installed successfully!")


In [ ]:
import os
import torch
import pickle
import json
import cv2
import zipfile
import shutil
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from google.colab import files

# **Step 1: Define Model Files Path (No Extraction Needed)**
model_dir = "/content/model_files"
model_weights_path = os.path.join(model_dir, "model_final.pth")
config_path = os.path.join(model_dir, "config.pkl")
metadata_path = os.path.join(model_dir, "metadata.json")

# **Step 2: Load Model Configuration**
if not os.path.exists(config_path):
    raise FileNotFoundError(f"❌ Config file not found: {config_path}")

with open(config_path, "rb") as f:
    cfg = pickle.load(f)

cfg.MODEL.WEIGHTS = model_weights_path  # Assign trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# **Step 3: Load Metadata**
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found: {metadata_path}")

with open(metadata_path, "r") as f:
    metadata_dict = json.load(f)

metadata = MetadataCatalog.get("my_custom_train_v2")
metadata.set(thing_classes=metadata_dict["thing_classes"])

# **Step 4: Load Model for Inference**
predictor = DefaultPredictor(cfg)
print("✅ Model loaded successfully!")

# **Step 5: Extract Test Images**
test_zip_path = "test.zip"
test_extract_dir = "./test_images"

if not os.path.exists(test_extract_dir):
    os.makedirs(test_extract_dir)

with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(test_extract_dir)

print(f"✅ Extracted test images to {test_extract_dir}")

# **Step 6: Run Inference on All Test Images**
output_dir = "./test_predicted"
os.makedirs(output_dir, exist_ok=True)

# Get all image files in the extracted folder
image_extensions = (".jpg", ".jpeg", ".png")
test_images = [f for f in os.listdir(test_extract_dir) if f.lower().endswith(image_extensions)]

if not test_images:
    print("❌ No valid images found in the test directory.")
else:
    print(f"✅ Found {len(test_images)} images. Running inference...")

    for img_name in test_images:
        img_path = os.path.join(test_extract_dir, img_name)
        output_img_path = os.path.join(output_dir, img_name)

        # Load image
        image = cv2.imread(img_path)

        if image is None:
            print(f"❌ Error loading {img_name}. Skipping...")
            continue

        # Perform inference
        outputs = predictor(image)

        # Visualize results
        v = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        output_img = out.get_image()[:, :, ::-1]

        # Save the output
        cv2.imwrite(output_img_path, output_img)
        print(f"✅ Saved prediction: {output_img_path}")

print(f"✅ All predictions saved in {output_dir}")

# **Step 7: Zip the Output Folder**
zip_output_path = "test_predicted.zip"
shutil.make_archive("test_predicted", 'zip', output_dir)
print(f"✅ Created zip file: {zip_output_path}")

# **Step 8: Download the Zip File**
files.download(zip_output_path)
print("✅ Downloading test_predicted.zip...")


FileNotFoundError: ❌ Config file not found: /content/model_files/config.pkl

# Train empty plates


In [ ]:
import os
import torch
import pickle
import json
import cv2
import zipfile
import random
import numpy as np
import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# **Step 1: Keep the Previous Training Data**
PREV_MODEL_DIR = "./model_files"  # Directory where the old model is saved
OUTPUT_DIR = "./output_finetuned"  # New output directory for fine-tuned model
os.makedirs(OUTPUT_DIR, exist_ok=True)

# **Step 2: Extract and Register the New Dataset**
data_zip_path = "/content/plate.zip"
data_extract_dir = "./plate_data"
annotations_file = "/content/labels_plates_segmentation_2025-02-27-02-52-56.json"

if not os.path.exists(data_extract_dir):
    os.makedirs(data_extract_dir)
    with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_extract_dir)
    print(f"✅ Extracted new dataset to {data_extract_dir}")

# Register dataset in Detectron2
register_coco_instances("plate_dataset", {}, annotations_file, data_extract_dir)
metadata = MetadataCatalog.get("plate_dataset")
dataset_dicts = DatasetCatalog.get("plate_dataset")

# **Step 3: Print Labeled Samples**
print(f"✅ Total labeled images: {len(dataset_dicts)}")
for d in random.sample(dataset_dicts, min(3, len(dataset_dicts))):  # Show 3 samples
    img_path = d["file_name"]

    if not os.path.exists(img_path):
        print(f"❌ Image not found: {img_path}. Skipping...")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ Error loading {img_path}. Skipping...")
        continue

    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    output_sample_path = f"./sample_{os.path.basename(img_path)}"
    cv2.imwrite(output_sample_path, vis.get_image()[:, :, ::-1])
    print(f"✅ Saved labeled sample: {output_sample_path}")

# **Step 4: Load the Pre-Trained Model for Fine-Tuning**
config_path = os.path.join(PREV_MODEL_DIR, "config.pkl")
model_weights_path = os.path.join(PREV_MODEL_DIR, "model_final.pth")

with open(config_path, "rb") as f:
    cfg = pickle.load(f)

cfg.DATASETS.TRAIN = ("plate_dataset",)
cfg.DATASETS.TEST = ()  # No testing during training
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Lower learning rate for fine-tuning
cfg.SOLVER.MAX_ITER = 500  # Fewer iterations for fine-tuning
cfg.MODEL.WEIGHTS = model_weights_path  # Load previous trained weights
cfg.OUTPUT_DIR = OUTPUT_DIR

# **Step 5: Train the Model on the New Dataset**
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# **Step 6: Save the Fine-Tuned Model**
fine_tuned_model_path = os.path.join(OUTPUT_DIR, "model_finetuned.pth")
torch.save(trainer.model.state_dict(), fine_tuned_model_path)
print(f"✅ Fine-tuned model saved at {fine_tuned_model_path}")


In [ ]:

# **Step 7: Run Inference on a Test Image**
test_image = "test_image.jpg"  # Change this to your actual test image path

# Check if test image exists
if not os.path.exists(test_image):
    print(f"❌ Test image not found: {test_image}")
else:
    image = cv2.imread(test_image)
    if image is None:
        print(f"❌ Error loading test image: {test_image}")
    else:
        predictor = DefaultPredictor(cfg)
        outputs = predictor(image)

        v = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        output_img_path = os.path.join(OUTPUT_DIR, "test_prediction.jpg")
        cv2.imwrite(output_img_path, out.get_image()[:, :, ::-1])
        print(f"✅ Saved inference output at {output_img_path}")

from google.colab import files

# Download the predicted image
if os.path.exists(output_img_path):
    files.download(output_img_path)
    print(f"✅ Downloading {output_img_path}...")
else:
    print("❌ No predicted image found to download.")


# Augmentations

In [ ]:
from detectron2.data import transforms as T

# Define a set of transformations
augmentations = T.AugmentationList([
    T.RandomFlip(horizontal=True, vertical=False),  # Flip horizontally
    T.RandomRotation(angle=[-15, 15]),  # Rotate between -15° and 15°
    T.RandomBrightness(0.8, 1.2),  # Adjust brightness
    T.RandomContrast(0.8, 1.2),  # Adjust contrast
    T.RandomSaturation(0.8, 1.2),  # Adjust saturation
])


In [ ]:
def custom_mapper(dataset_dict):
    dataset_dict = dataset_dict.copy()  # Make a copy
    image = cv2.imread(dataset_dict["file_name"])  # Load image

    # Apply augmentations
    aug_input = T.AugInput(image)
    transform = augmentations(aug_input)
    image = aug_input.image  # Get augmented image

    # Apply transformations to annotations
    for anno in dataset_dict["annotations"]:
        anno["bbox"] = transform.apply_box(anno["bbox"])

    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1))
    return dataset_dict


In [ ]:
from detectron2.data import DatasetMapper
from detectron2.data.build import build_detection_train_loader

cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True  # Ignore images without objects
train_loader = build_detection_train_loader(cfg, mapper=custom_mapper)
